In [1]:
from datetime import datetime
import json
import os
import numpy as np

import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
from keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Dense
import matplotlib.pyplot as plt

from utility.train_data_loader import load_train_data

Using TensorFlow backend.


In [2]:
epochs = 1
batch_size = 64
specialization = "beauty"

In [3]:
categories_file = open("../data/categories.json", "r")
categories = json.load(categories_file)


In [4]:
all_subcategories = {k.lower(): v for k, v in categories['Mobile'].items()}
all_subcategories.update({k.lower(): v for k, v in categories['Fashion'].items()})
all_subcategories.update({k.lower(): v for k, v in categories['Beauty'].items()})


In [5]:
data_root = "../../"+specialization+"_image/"


In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [7]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2"


In [8]:
trainData = load_train_data()
testData = pd.read_csv("../data/test.csv")

custom train data used


In [9]:
train_data_specialized = trainData[trainData['image_path'].str.contains(specialization)][::10]
train_data_specialized['image_path'] = train_data_specialized['image_path'].\
    map(lambda x: x.lstrip(specialization+'_image/'))


In [10]:
validation_data_specialized = trainData[trainData['image_path'].str.contains(specialization)][::100]
validation_data_specialized['image_path'] = validation_data_specialized['image_path'].\
    map(lambda x: x.lstrip(specialization+'_image/'))



In [11]:
test_data_specialized = testData[testData['image_path'].str.contains(specialization)][:]
test_data_specialized['image_path'] = test_data_specialized['image_path'].\
    map(lambda x: x.lstrip(specialization+'_image/'))


In [12]:
print(test_data_specialized.shape)

(76545, 3)


In [15]:
print(test_data_specialized)

           itemid                                              title  \
0       370855998               flormar 7 white cream bb spf 30 40ml   
1       637234604  maybelline clear smooth all in one bb cream sp...   
2       690282890  murah innisfree eco natural green tea bb cream...   
3       930913462  loreal white perfect day cream spf 17 pa white...   
4      1039280071  hada labo cc cream ultimate anti aging spf 35 ...   
5      1327710392  cathy doll cc speed white powder pact spf 40 o...   
6      1328802799           safi white natural brightening cream 45g   
7      1330468145  light beige 03 bioaqua bb cushion exquisite de...   
8      1677309730  new produk missha m perfect bb cream share in ...   
9      1683142205      ready laneige bb cushion anti aging spf 50 pa   
10     1704308849  terupdate laneige bb cushion anti aging spf 50...   
11     1736407742  new collection bioaqua cushion blush on smooth...   
12     1747613934  blp beauty face powder light beige preloved l

In [16]:
i=0
for index, row in test_data_specialized.iterrows():    
    if not os.path.isfile(os.path.join(data_root,row['image_path'])):
        print(row)
print(i)

itemid                                               1327710392
title         cathy doll cc speed white powder pact spf 40 o...
image_path                  1e50828d5594721a7d5d5c1ff78afbd.jpg
Name: 5, dtype: object
itemid                                               1793784525
title         big discount set garnier sakura white pinkish ...
image_path                   836486176dd903cf07425ff71ad797.jpg
Name: 13, dtype: object
itemid                                                728370998
title         compact powder viva kuning langsat pengantin n...
image_path                  2e3487c251f127f59ffb8b0e7079ad2.jpg
Name: 19, dtype: object
itemid                                   1760895527
title         dr pure white natural lightning cream
image_path      0f9a0875b2a9ca0a20de2c7ddd798c8.jpg
Name: 28, dtype: object
itemid                                               1078543239
title         sale bedak wajah ponds magic powder bb pinkish...
image_path                  383f69e64c47f0dda

itemid                                               1601504180
title         terbukti refill maybelline bb super cushion sp...
image_path                  2fd015d6f54d6e0902bfe5a9c249060.jpg
Name: 600, dtype: object
itemid                                     1713440511
title         compact powder kuning natural theraskin
image_path        4b287e163b7314798eda3f425e2d92c.jpg
Name: 602, dtype: object
itemid                                               1742920166
title         cantik missha m perfect cover bb cream 50ml li...
image_path                  6be1a0b9fc16ea476b4d409a3ea5cb4.jpg
Name: 603, dtype: object
itemid                                         56567291
title         cathy doll speed white cc cream spf 50 pa
image_path          f7bbc3518232d5cf67148db4a92f1f7.jpg
Name: 606, dtype: object
itemid                                                 63528206
title         theraskin compact powder kn kuning natural bed...
image_path                   99c678de7cffe29d85a1a4d86f71e

Name: 1204, dtype: object
itemid                                               1705342406
title         new 100 ori pure shine cc cushion nature repub...
image_path                  64ca0c463692c426d1e684a0247d0e2.jpg
Name: 1208, dtype: object
itemid                                  942342173
title            zoya natural white bb cream 20ml
image_path    61bc1952b44e49449ce2ac2abbcd37e.jpg
Name: 1215, dtype: object
itemid                                               1603170908
title         murah di shopee laneige bb cushion whitening 2...
image_path                  31351113e0e70d98ca5a9c9902cd4bd.jpg
Name: 1220, dtype: object
itemid                                               1615316862
title         safi white expert cc cream spf 24pa whitening ...
image_path                  c557234b96462d4d084142932264ed3.jpg
Name: 1221, dtype: object
itemid                                    1681797963
title         safi white expert cc cream spf 24 20gr
image_path       6deca08f4b025c9e35f59

Name: 1730, dtype: object
itemid                                               1628998046
title         langsung kirim innisfree water fit cover cushi...
image_path                  1fc72f302a0524755df255178d4ebb2.jpg
Name: 1731, dtype: object
itemid                                               1727599632
title         gaya ysl top secret cc cream colour corrector ...
image_path                  38bf9b6257b718f72c898bed4a305de.jpg
Name: 1734, dtype: object
itemid                                       1794098787
title         oam29 olay saudi arab natural white cream
image_path          5f3acbe876e2118f133ba5bd59c39ed.jpg
Name: 1739, dtype: object
itemid                                                705844850
title         exp 2020 sisa stok laneige bb cushion pore con...
image_path                  f94675c08384b73fbb9cd1ea68b8a7f.jpg
Name: 1743, dtype: object
itemid                                               1779872291
title         dijual lanore whitening anti aging cc cream li..

Name: 2292, dtype: object
itemid                                                707905352
title         maybelline white superfresh spf 34 pa long las...
image_path                  4fc0a599591da7fb5c5a0173e5493eb.jpg
Name: 2293, dtype: object
itemid                                               1485686808
title         stok terakhir missha m magic cushion spf50 pa ...
image_path                  4009c73f81583d541904283ab294fc7.jpg
Name: 2302, dtype: object
itemid                                                187373688
title         diskon etude house any cushion color corrector...
image_path                  19199eb02dddd52ca3e3c9ff59a1872.jpg
Name: 2310, dtype: object
itemid                                               1155997444
title         new collection innisfree skinny coverfit cushi...
image_path                   6a9fc0678b22f7edc81de381ff2477.jpg
Name: 2315, dtype: object
itemid                                     1556027027
title         laneige bb cushion anti aging spf 

...

Name: 73958, dtype: object
itemid                                  176330610
title                            wardah lip cream
image_path    fa6bd6af3a70ab91978bf262f59ea82.jpg
Name: 73960, dtype: object
itemid                                   198977731
title         pixi mattelustre lipstick petal pink
image_path      7947e761294a12c6db7f03204f18d7.jpg
Name: 73962, dtype: object
itemid                                  204472186
title                          lip cream makeover
image_path    28912d3f6710324849165c7675da0e4.jpg
Name: 73965, dtype: object
itemid                                           210189553
title         zoya matte lip paint lengkap semua warna ada
image_path             50abe0eebb63fe09f3cf3bfab1e23f6.jpg
Name: 73970, dtype: object
itemid                                        213224366
title         giordani gold iconic matte lipstck spf 12
image_path          2f93fbb297f2531120ff5ad1366df5a.jpg
Name: 73971, dtype: object
itemid                                  

itemid                                                170560467
title         she loves summer matte 24h lipgloss lipcream c...
image_path                  ca0b7974c7ea661f0ef766f2ca1f77d.jpg
Name: 74528, dtype: object
itemid                                  198960398
title                  mizzu lip cream valipcious
image_path    207291e1ad0bb690781757f73f195f1.jpg
Name: 74533, dtype: object
itemid                                272794849
title              purbasari hi matte lip cream
image_path    c10f05530889ecb5cf3a2f1f8f58e.jpg
Name: 74543, dtype: object
itemid                                                368471078
title         box silver isi 12 kylie holiday edition matte ...
image_path                  449e28490f7315727b5a8bdd9ed8b0c.jpg
Name: 74557, dtype: object
itemid                                  480912935
title         barbie lips cream bpom ertos erto s
image_path        8b7d262e6903f5746490714a455.jpg
Name: 74570, dtype: object
itemid                               

itemid                                                794013299
title         hot bds281217 hijau m.n mn menow eye lip liner...
image_path                  52c46dfd2dd893deef6fe6e72c4bef3.jpg
Name: 75151, dtype: object
itemid                                               800027937
title         maybelline the powder mattes shade touch of nude
image_path                 fd427f20018b9d330e81c15f2d895bf.jpg
Name: 75152, dtype: object
itemid                                                805545487
title         promo terbatas 1 box isi 12 pc kkw kylie liqui...
image_path                  9efc472838fe12826b66a08cea58283.jpg
Name: 75154, dtype: object
itemid                                          841091553
title         golden rose velvet matte 4 2g flormar cream
image_path              241da197284c0b46d4aa50ff5ab6e.jpg
Name: 75159, dtype: object
itemid                                       850880897
title         sale wardah lipcream matte 01 red dicted
image_path         c91e11b1bb6771af

itemid                                 1186118868
title           etude dear darling tint ice cream
image_path    c777408df50bcf25f87685acdeed4f2.jpg
Name: 75740, dtype: object
itemid                                 1215139686
title                            lip cream wardah
image_path    fd7a7db79cce3af7606421f20a6d574.jpg
Name: 75743, dtype: object
itemid                                      1215834163
title         the one express lip crayon oriflame asli
image_path         f35b0da5c823fbc9a94aa610d5ca16c.jpg
Name: 75744, dtype: object
itemid                                               1218488522
title         termurah original lipstick maybelline powder m...
image_path                  69b89977df04fc73e600bc811b9db53.jpg
Name: 75745, dtype: object
itemid                               1259121110
title                 love 3ce glossy lip stick
image_path    026f0e9ed84ed1228de4e674e58c0.jpg
Name: 75752, dtype: object
itemid                                 1268702705
title         

itemid                                               1202980522
title         hot sale kylie liquid box glamorous silky lipg...
image_path                  20eb5753537ce3d1ab28f8c4d190a71.jpg
Name: 76246, dtype: object
itemid                                               1204729965
title         bayar di tempat babybear lipstick matte velvet...
image_path                  f9cee44cb2681f6684a5b6d3c801c06.jpg
Name: 76247, dtype: object
itemid                                               1252038010
title         bayar di tempat bluelans lipstick cair matte b...
image_path                  13070bc5b092cfca3524b9e6430edf0.jpg
Name: 76253, dtype: object
itemid                                    1285550228
title         hello lipstick pensil matte tahan lama
image_path       576be9b09da0506a43aedd9077301bf.jpg
Name: 76261, dtype: object
itemid                                               1324362438
title         termurah rollover reaction sueded lip and chee...
image_path                  1

In [24]:
categories_specialized = {k.lower(): v for k, v in categories[specialization.capitalize()].items()}


In [25]:
categories_specialized

{'bb & cc cream': 5,
 'blush on': 2,
 'bronzer': 11,
 'concealer': 7,
 'contour': 6,
 'face palette': 0,
 'foundation': 1,
 'highlighter': 8,
 'lip gloss': 14,
 'lip liner': 15,
 'lip tint': 13,
 'lipstick': 12,
 'other face cosmetics': 4,
 'other lip cosmetics': 16,
 'powder': 3,
 'primer': 9,
 'setting spray': 10}

In [26]:
IMAGE_SIZE = hub.get_expected_image_size(hub.Module(feature_extractor_url))


INFO:tensorflow:Using C:\Users\HANSTA~1\AppData\Local\Temp\tfhub_modules to cache modules.


In [27]:
image_generator = datagen.flow_from_dataframe(train_data_specialized,
                                              directory=os.path.join(data_root),
                                              x_col="image_path",
                                              y_col="item_category",
                                              target_size=IMAGE_SIZE,
                                              color_mode="rgb",
                                              class_mode="categorical",
                                              shuffle=True,
                                              batch_size=64,
                                              )

Found 23209 images belonging to 17 classes.


In [28]:
valid_generator = valid_datagen.flow_from_dataframe(validation_data_specialized,
                                                    directory=os.path.join(data_root),
                                                    x_col="image_path",
                                                    y_col="item_category",
                                                    target_size=IMAGE_SIZE,
                                                    color_mode="rgb",
                                                    class_mode="categorical",
                                                    shuffle=True,
                                                    batch_size=64,
                                                    )

Found 2332 images belonging to 17 classes.


In [29]:
test_generator = test_datagen.flow_from_dataframe(test_data_specialized,
                                                   directory=os.path.join(data_root),
                                                   x_col="image_path",
                                                  y_col=None,
                                                  class_mode=None,
                                                  shuffle=False,
                                                   target_size=IMAGE_SIZE,
                                                   color_mode="rgb",
                                                   batch_size=64,
                                                   )

Found 62059 images.


In [35]:
label_names = sorted(image_generator.class_indices.items(), key=lambda pair:pair[1])
label_names = np.array([key.title() for key, value in label_names])


In [36]:
def feature_extractor(x):
    feature_extractor_module = hub.Module(feature_extractor_url)
    return feature_extractor_module(x)

In [37]:
model = Sequential()
model.add(Lambda(feature_extractor, input_shape=IMAGE_SIZE+[3], trainable=True))
model.add(Dense(len(categories_specialized), activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                21777     
Total params: 21,777
Trainable params: 21,777
Non-trainable params: 0
_________________________________________________________________


In [38]:
def gen_filename_h5():
    return 'epoch_'+str(epochs) + '_' + datetime.now().strftime("%m_%d_%Y_%H_%M_%S")


In [67]:
def gen_filename_csv():
    return 'epoch_'+str(epochs) + '_' + str(max_words) + '_' + datetime.now().strftime("%m_%d_%Y_%H_%M_%S")



In [39]:
filepath = "../checkpoints/"+gen_filename_h5()+"v2.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [50]:
steps_per_epoch = image_generator.samples//image_generator.batch_size
valid_steps_per_epoch = valid_generator.samples // valid_generator.batch_size
test_steps_per_epoch = test_generator.samples // test_generator.batch_size

In [43]:
history = model.fit_generator(generator=image_generator,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=valid_generator,
                    validation_steps=valid_steps_per_epoch,
                    epochs=epochs,
                    callbacks=[checkpointer],
                    )

Epoch 1/1
362/362 [==============================] - 95s 263ms/step - loss: 1.6395 - acc: 0.4854 - val_loss: 1.4611 - val_acc: 0.5428

Epoch 00001: val_acc improved from -inf to 0.54277, saving model to ../checkpoints/epoch_25_03_14_2019_00_47_25v2.hdf5


In [45]:
def plot_history(history):
    plt.style.use('ggplot')
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()


In [46]:
plot_history(history)

In [51]:
prediction_specialized = model.predict_generator(test_generator, verbose=1, steps = test_steps_per_epoch)


969/969 [==============================] - 364s 376ms/step


In [59]:
predicted_label_specialized = [np.argmax(prediction_specialized[i]) for i in range(len(prediction_specialized))]


In [68]:
df = pd.DataFrame({'itemid': test_data_specialized['itemid'].astype(int), 'Category': predicted_label_specialized})


ValueError: array length 62016 does not match index length 76545